Agenda in this project
1. Run a hyper-parameter sweep on a Deep learning training Script
2. Compare the results of the run in the MLFlow UI
3. Choose the best Run amd register it as a model
4. Deploy the model to a REST API
5. Buils a container image suitable for deployment to a cloud platform

In [1]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe #This is used to do Hyper-Parameter tunning in Deep learning environment
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

2024-11-09 13:54:48.439481: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731140688.518879   19051 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731140688.539725   19051 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-09 13:54:48.747615: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#  Load the dataSet
data = pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";"
)
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [3]:
# Split the data into training, validation and test
train,test = train_test_split(data,test_size=0.25,random_state=42)

In [4]:
train_x = train.drop(['quality'], axis=1).values #.values to convert it into array
train_y = train['quality'].values.ravel()

# test DataSet
test_x =  test.drop(['quality'], axis=1).values
test_y = test['quality'].values.ravel()

# Splitting this train data into train and validation data
train_x,valid_x,train_y, valid_y = train_test_split(train_x,train_y,test_size=0.20,random_state=42)

In [5]:
signature = infer_signature(train_x, train_y)

In [6]:
# ANN Model
def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):
    # Defining model Archtecture
    mean = np.mean(train_x, axis=0) # axis = 0 means calculating mean column vise
    var = np.var(train_x, axis=0)

    model = keras.Sequential(
        [
            keras.Input(shape=(train_x.shape[1],)), #train_x.shape[1] = getting number of columns
            keras.layers.Normalization(mean = mean, variance=var),
            keras.layers.Dense(64, activation='relu'),
            keras.layers.Dense(1)
        ]
    )

    # Compile the model
    model.compile(optimizer = keras.optimizers.SGD(
        # Here we will do experiment with values of learning rate and momentum and params will have a 
        # list of lr and momentum and will track every experiment with MLFlow
        learning_rate = params["lr"],
        momentum = params["momentum"] 
    ),
    loss = "mean_squared_error",
    metrics = [keras.metrics.RootMeanSquaredError()]
    )

    # Train the ANN model with lr and momentum params with MLFlow Tracking
    with mlflow.start_run(nested=True):
        model.fit(train_x,train_y,validation_data=(valid_x,valid_y),
                  epochs=epochs,
                  batch_size = 32,
                  )
        
        # Evaluate the model

        eval_result = model.evaluate(valid_x,valid_y,batch_size=32)

        eval_rsme = eval_result[1]

        # Now i want to track this eval_rsme value and all params values
        # Now log the params and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse",eval_rsme)

        # Log the model
        mlflow.tensorflow.log_model(model,"model",signature=signature)

        return {"loss":eval_rsme, "status":STATUS_OK, "model":model}


In [7]:
def objective(params):
    # MLFlow will track the parameters and results for each run
    result = train_model(
        params=params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )

    return result

In [8]:
space = {
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-1)), #here we are defining range of learning rate from 1e-5 to 1e-1
    "momentum":hp.uniform("momentum",0.0,1.0)
}

In [10]:
mlflow.set_experiment("/wine-quality")
with mlflow.start_run():
    # Conduct the hyper-Parameter search using hyperopt
    trials = Trials()
    best = fmin(
        fn = objective,
        space = space,
        algo = tpe.suggest,
        max_evals=4,
        trials = trials
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rsme",best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"],"model",signature=signature)

    print(f"Best Parameters: {best}")
    print(f"Best eval rsme {best_run['loss']}")

Epoch 1/3                                            

 1/92 ━━━━━━━━━━━━━━━━━━━━ 34s 375ms/step - loss: 36.9697 - root_mean_squared_error: 6.0822
 2/92 ━━━━━━━━━━━━━━━━━━━━ 0s 693us/step - loss: 36.9926 - root_mean_squared_error: 6.0822
 3/92 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 36.6296 - root_mean_squared_error: 6.0521
 4/92 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 36.2671 - root_mean_squared_error: 6.0219
 5/92 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 36.0043 - root_mean_squared_error: 5.9999
 6/92 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 35.7053 - root_mean_squared_error: 5.9748   
58/92 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 25.9371 - root_mean_squared_error: 5.0447
57/92 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 26.0884 - root_mean_squared_error: 5.0292
59/92 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 25.7879 - root_mean_squared_error: 5.0447
60/92 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 25.6403 - root_mean_squared_error: 5.0138
61/92 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 

KeyboardInterrupt: 

In [26]:
import tensorflow as tf
print(tf.__version__)


2.18.0
